# Housing Regression Kaggle Competition
This is a notebook for my submissions based on the following [Kaggle Competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques). 

Given 79 features about residential homes in Iowa (as well as their sale prices), the goal is to predict the price of future homes. 

## Data
We are given two files. $\texttt{train.csv}$ from which our model is trained on, and $\texttt{test.csv}$ from which we make predictions that are submitted. 

## Evaluation
From the list of predicted house prices, our predictions will be scored by taking the RMSE between the logarithm of the predicted sales price and the logarithm of the observed sales price.

## Other
This notebook is meant to be a condensed version of all the work. All files used can be found [here](https://github.com/fattorib/HousingRegression).